In [ ]:
#Optional: upload the train/test data to Google Colab. if you use it.
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/My Drive/input_data.zip" -d /content

Mounted at /content/drive


In [1]:
!pip install kaggle

In [4]:
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle datasets download -d klausmikaelson2002/solafune-mining-classification-dataset

100% 12.3G/12.3G [02:18<00:00, 172MB/s]
100% 12.3G/12.3G [02:19<00:00, 95.3MB/s]


In [6]:
!pip install pytorch_lightning torchmetrics wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00


In [7]:
!unzip -q "/content/solafune-mining-classification-dataset.zip" -d /content

In [8]:
import pytorch_lightning as pl
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
import numpy as np
import pandas as pd
import tifffile as tiff
from torchmetrics import F1Score
import os
import csv
from tqdm.notebook import tqdm
import random
import matplotlib.pyplot as plt
from types import SimpleNamespace

import wandb
from pytorch_lightning.loggers import WandbLogger

os.environ["WANDB_MODE"] = "online"
from google.colab import userdata
userdata.get('wandb_key') #Remember to input your W&B key onto the Secrets if you use it.

debug = False

config = {
    "batch_size" : 16,
    "num_workers": os.cpu_count(),
    "img_size" : 384,
    "max_epochs" : 3 if debug else 20,
}

config = SimpleNamespace(**config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(seed=42)

In [10]:
def load_and_convert_tiff(file_path):
    image = tiff.imread(file_path)
    R = image[:, :, 3]*255*2
    G = image[:, :, 2]*255*2
    B = image[:, :, 1]*255*2
    rgb_image = np.stack((R, G, B), axis=2).astype(np.uint8)
    rgb_image = Image.fromarray(rgb_image)
    return rgb_image

In [11]:
if debug:
    pos_tiff_files = [
        '/content/train/train/train_3.tif',
        '/content/train/train/train_12.tif',
        '/content/train/train/train_14.tif',
        '/content/train/train/train_15.tif',
    ]
    plt.figure(figsize=(8, 8))
    plt.suptitle('Positives')
    for i, file_path in enumerate(pos_tiff_files):
        rgb_image = load_and_convert_tiff(file_path)
        plt.subplot(2, 2, i+1)
        plt.imshow(rgb_image)
        plt.axis('off')
    plt.show()

    neg_tiff_files = [
        '/content/train/train/train_0.tif',
        '/content/train/train/train_1.tif',
        '/content/train/train/train_2.tif',
        '/content/train/train/train_4.tif',
    ]
    plt.figure(figsize=(8, 8))
    plt.suptitle('Negatives')
    for i, file_path in enumerate(neg_tiff_files):
        rgb_image = load_and_convert_tiff(file_path)
        plt.subplot(2, 2, i+1)
        plt.imshow(rgb_image)
        plt.axis('off')
    plt.show()

In [17]:
#Dataset class
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transforms.Compose([
            transforms.Resize(config.img_size),
            transforms.RandomHorizontalFlip(p=0.5),# extra
            transforms.RandomVerticalFlip(p=0.5), # extra
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = load_and_convert_tiff(img_path)
        label = self.img_labels.iloc[idx, 1]
        image = self.transform(image)
        return image, label

def create_datasets(csv_file, img_dir, val_split=0.1):
    full_dataset = CustomImageDataset(csv_file=csv_file, img_dir=img_dir)
    labels = full_dataset.img_labels.iloc[:, 1].tolist()
    train_idx, val_idx = train_test_split(
        range(len(labels)),
        test_size=val_split,
        stratify=labels,
        random_state=42,
    )

    train_dataset = Subset(full_dataset, train_idx)
    val_dataset = Subset(full_dataset, val_idx)

    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]
    print("train_size:", len(train_dataset), "with pos:", train_labels.count(1))
    print("val_size:", len(val_dataset), "with neg:", val_labels.count(1))

    return train_dataset, val_dataset

train_dataset, val_dataset = create_datasets(csv_file='/content/train/answer.csv', img_dir='/content/train/train')

train_size: 1116 with pos: 230
val_size: 125 with neg: 26


In [18]:
# Model
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.model = torch.hub.load('hankyul2/EfficientNetV2-pytorch', 'efficientnet_v2_s', pretrained=True, nclass=2)
        self.f1 = F1Score(num_classes=2, task='binary')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        assert y.min() >= 0 and y.max() <= 1, f"Labels out of range: min={y.min()}, max={y.max()}"
        logits = self(x)

        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)

        f1_score = self.f1(preds, y)
        self.log('train_loss', loss)
        self.log('train_f1', f1_score, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        assert y.min() >= 0 and y.max() <= 1, f"Labels out of range: min={y.min()}, max={y.max()}"

        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)

        f1_score = self.f1(preds, y)
        self.log('val_loss', loss)
        self.log('val_f1', f1_score, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def train_dataloader(self):
        dataloader = DataLoader(train_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=True)
        return dataloader

    def val_dataloader(self):
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=False)
        return val_loader

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-5)
        #optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-3)
        return optimizer

In [19]:
model = ImageClassifier()
train_dataloader = model.train_dataloader()
if debug:
    trainer = pl.Trainer(max_epochs=config.max_epochs, log_every_n_steps=1)
    trainer.fit(model, train_dataloader)
else:
    wandb.init(project='solafune-mining-colab-2')
    wandb_logger = WandbLogger()
    trainer = pl.Trainer(max_epochs=config.max_epochs, logger=wandb_logger) #, log_every_n_steps=20
    trainer.fit(model, train_dataloader)
    run_name = wandb.run.name
    wandb.finish()
    torch.save(model.state_dict(), f'my_model_{run_name}.pth')

Using cache found in /root/.cache/torch/hub/hankyul2_EfficientNetV2-pytorch_main
wandb: Currently logged in as: ahirwarnirmal2017. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:390: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type           | Params
-----------------------------------------
0 | model | EfficientNetV2 | 20.2 M
1 | f1    | BinaryF1Score  | 0     
-----------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_f1,▁▃▃▄▅▆▆▇▇▇██▇▇█▇████
train_loss,█▇▇▆▆▄▃▃▂▂▂▂▂▄▃▁▂▂▂▂▂▂▂▁▂▂▁▃
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_f1,▁▃▃▅▇▇▄▅█▃▄▅▅█▄█▆▅▅▇
val_loss,█▆▅▅▄▃▃▂▂▂▂▁▁▁▂▁▂▂▁▁
epoch,19
train_f1,0.96761
train_loss,0.22988
trainer/global_step,1399
val_f1,0.87282


In [20]:
#Inference
class InferenceDataset(Dataset):
    def __init__(self, img_dir,file_names):
        self.img_dir = img_dir
        self.img_names = file_names
        self.transform = transforms.Compose([
            transforms.Resize(config.img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = load_and_convert_tiff(img_path)
        image = self.transform(image)
        return image, self.img_names[idx]

file_names = []
with open('/content/uploadsample.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        file_names.append(row[0])

inference_dataset = InferenceDataset(img_dir='/content/evaluation_images', file_names=file_names)
inference_loader = DataLoader(inference_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=False)

In [ ]:
model.eval()
model.to(device)
with open(f'my_answer_{run_name}.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    predictions = []
    file_names = []
    with torch.no_grad():
        for data, file_paths in tqdm(inference_loader):
            data = data.to(device)
            logits = model(data)
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            file_names.extend(file_paths)

    combined = list(zip(file_names, predictions))
    for file_name, prediction in combined:
        writer.writerow([file_name, prediction])

print(predictions.count(1), "positives")

  0%|          | 0/78 [00:00<?, ?it/s]